In [1]:
from selenium import webdriver
from tqdm.notebook import tqdm
from time import sleep
import pandas as pd
import re

In [12]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://music.yandex.ru/non-music/category/scienceandeducation_science/albums')

In [13]:
podcasts = pd.read_excel('podcasts_preprocessed.xlsx')
podcasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          103 non-null    int64 
 1   Unnamed: 0.1        103 non-null    int64 
 2   Header              103 non-null    object
 3   Likes               103 non-null    int64 
 4   Link                103 non-null    object
 5   Number of Episodes  103 non-null    int64 
 6   Description         103 non-null    object
dtypes: int64(4), object(3)
memory usage: 5.8+ KB


In [14]:
podcasts = podcasts.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [15]:
episodes = pd.DataFrame(columns=['Title', 'Podcast', 'Duration', 'Link'])

In [16]:
# durations вроде рабочие
def get_episodes_metadata(podcasts_df):
    
    global episodes
    
    for link in podcasts_df['Link']:

        driver.get(link)
        screen_height = driver.execute_script("return window.screen.height;")
        i = 0
        link_index = podcasts_df.index[podcasts_df['Link'] == link][0]
        podcast = podcasts_df['Header'][link_index]
        titles, podcasts, durations, links = [], [], [], []
        
        while True:
            
            driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});") 
            i += 1
            sleep(0.5)
            
            title_elements = driver.find_elements_by_xpath('//a[@class="d-track__title deco-link deco-link_stronger line-clamp"]')
            titles = [title_elements[i].text for i in range(len(title_elements)) if title_elements[i].text not in titles]
            
            duration_elements = driver.find_elements_by_xpath('//span[@class="typo-track deco-typo-secondary"]')
            durations = [duration_elements[i].text for i in range(len(duration_elements))]
            
            link_elements = driver.find_elements_by_xpath('//*[contains(@href,"/track/")]')
            links = [link_elements[i].get_attribute('href') for i in range(len(link_elements))]
            
            scroll_height = driver.execute_script("return document.body.scrollHeight;")  
            if screen_height * i > scroll_height:
                break
        
        for i in range(len(titles)):
       
            ep = {'Title': titles[i], 
                  'Podcast': podcast, 
                  'Duration': durations[i],
                  'Link': links[i]}

                   
            episodes = episodes.append(ep, ignore_index=True)
        
        print(f'{link_index + 1} out of {podcasts_df.shape[0]} done, {len(titles)} episodes added')


In [17]:
get_episodes_metadata(podcasts)

C:\Users\Daria\AppData\Local\Temp/ipykernel_12688/1118650448.py:21: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  title_elements = driver.find_elements_by_xpath('//a[@class="d-track__title deco-link deco-link_stronger line-clamp"]')
C:\Users\Daria\AppData\Local\Temp/ipykernel_12688/1118650448.py:24: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  duration_elements = driver.find_elements_by_xpath('//span[@class="typo-track deco-typo-secondary"]')
C:\Users\Daria\AppData\Local\Temp/ipykernel_12688/1118650448.py:27: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  link_elements = driver.find_elements_by_xpath('//*[contains(@href,"/track/")]')


1 out of 103 done, 17 episodes added
2 out of 103 done, 102 episodes added
3 out of 103 done, 103 episodes added
4 out of 103 done, 15 episodes added
5 out of 103 done, 44 episodes added
6 out of 103 done, 57 episodes added
7 out of 103 done, 103 episodes added
8 out of 103 done, 103 episodes added
9 out of 103 done, 102 episodes added


KeyboardInterrupt: 

In [19]:
episodes[episodes['Podcast'] == 'The Big Beard Theory']

,Title,Podcast,Duration,Link
544,102: Новая космическая гонка к Луне,The Big Beard Theory,29:43,https://music.yandex.ru/album/6408431/track/50...
545,101: Открытие NASA и граница допустимого кликб...,The Big Beard Theory,26:43,https://music.yandex.ru/album/6408431/track/47...
546,100: Путешествия во времени,The Big Beard Theory,31:42,https://music.yandex.ru/album/6408431/track/47...
547,99: Живем ли мы в симуляции,The Big Beard Theory,31:03,https://music.yandex.ru/album/6408431/track/47...
548,98: Что такое квантовая телепортация,The Big Beard Theory,63:46,https://music.yandex.ru/album/6408431/track/47...
...,...,...,...,...
641,"5: SpaceX, блестки на службе NASA и трансплант...",The Big Beard Theory,17:52,https://music.yandex.ru/album/6408431/track/47...
642,4: Частная космонавтика с Зеленым котом,The Big Beard Theory,129:22,https://music.yandex.ru/album/6408431/track/47...
643,"3: БАК, черные дыры и капсуляция цивилизаций",The Big Beard Theory,21:59,https://music.yandex.ru/album/6408431/track/47...
644,2: Космические телескопы,The Big Beard Theory,23:08,https://music.yandex.ru/album/6408431/track/50...


# DRAFTS

In [ ]:
def get_titles_with_ids(podcasts_df):
    
    global episodes
    
    for link in podcasts_df['Link']:
        driver.get(link)
        screen_height = driver.execute_script("return window.screen.height;")
        i = 0
        titles = []
        link_index = podcasts_df.index[podcasts_df['Link'] == link][0]
        podcast = podcasts_df['Header'][link_index]
        
        while True:
            driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});") 
            i += 1
            sleep(1)
            title_elements = driver.find_elements_by_xpath('//a[@class="d-track__title deco-link deco-link_stronger line-clamp"]')
            ep_id = driver.find_elements_by_xpath('//')
            titles.extend([f'{title_elements[i].text}_' for i in range(len(title_elements)) if title_elements[i].text not in titles])
    
            scroll_height = driver.execute_script("return document.body.scrollHeight;")  
            if (screen_height) * i > scroll_height:
                break

In [23]:
driver.get('https://music.yandex.ru/album/7121797')
durations = driver.find_elements_by_xpath('')
for d in tqdm(durations):
    print(d.text)

[ class="" data-b="2078" data-id="1"><div class="d-track__start-column"><div class="entity-cover deco-entity-image-placeholder d-track__cover entity-cover_size_Large"><img class="entity-cover__image deco-pane" src="//avatars.yandex.net/get-music-content/4747389/1d0167a1.a.7121797-4/50x50" srcset="//avatars.yandex.net/get-music-content/4747389/1d0167a1.a.7121797-4/50x50, //avatars.yandex.net/get-music-content/4747389/1d0167a1.a.7121797-4/100x100 2x"></div><div class="d-track__bubble"></div><div class="d-track__play d-track__hover"><span class="d-button-play__wrap" data-b="2079"><button class="button-play button button_round button_action button_ico local-icon-theme-white button-play__type_track" data-b="2081" data-idx="98195516"><span class="button-inner deco-button-stylable"><span class="d-icon deco-icon d-icon_play-small  "></span></span></button></span></div></div><div class="d-track__overflowable-column">    <div class="d-track__overflowable-wrapper deco-typo-secondary block-layout"> <div class="d-track__name" title="Новогодняя программа - Сергей Виватенко отвечает на вопросы слушателей."> <a href="/album/7121797/track/98195516" class="d-track__title deco-link deco-link_stronger line-clamp"> Новогодняя программа - Сергей Виватенко отвечает на вопросы слушателей. </a></div><div class="d-track__meta"><span class="d-track__artists"></span></div></div><div class="d-track__info-column"> </div><div class="d-track__end-column"><div class=" d-track__actions"> <span class="d-like d-like_theme-track deco-track-like" data-b="2083"><span class="d-like__icon-wrapper deco-track-like-icon-wrapper"><span class="d-icon deco-icon d-icon_heart  d-like__icon deco-buttonish-like-icon-bright" title="Сделайте рекомендации точнее и добавьте выпуск в раздел «Моя коллекция»"></span><span class="d-icon deco-icon d-icon_heart  d-like__icon deco-buttonish-like-icon-pale" title="Сделайте рекомендации точнее и добавьте выпуск в раздел «Моя коллекция»"></span></span></span><span class="dislike deco-track-dislike dislike_theme-track dislike__track-hover-hidden" data-b="2084" "=""><span class="dislike__icon-wrapper deco-track-dislike-icon-wrapper"><span class="d-icon deco-icon d-icon_circle-crossed_track  dislike__icon deco-buttonish-dislike-icon-bright" title="Нажмите, и мы не будем больше рекомендовать этот выпуск"></span><span class="d-icon deco-icon d-icon_circle-crossed_track  dislike__icon deco-buttonish-dislike-icon-pale" title="Нажмите, и мы не будем больше рекомендовать этот выпуск"></span></span></span><div class="d-tooltip deco-tooltip   popup_hidden popup deco-pane-popup d-tooltip_min-width" data-b="2086"><div class="deco-tooltip__pointer-before"></div><div class="d-tooltip__stair"></div><div class="d-tooltip__content deco-tooltip"><div class="d-tooltip__message">Войдите, и мы запомним, что вам не нравится этот выпуск</div>   <div class="d-tooltip__actions"> <button class="d-button deco-button deco-button-action local-icon-theme-white d-button_size_S d-tooltip__btn-agree" data-b="2087" type="button"><span class="d-button-inner deco-button-stylable"><span class="d-button__inner"><span class="d-button__label">Войти</span></span></span></button> </div> </div></div></div><div class="d-context-menu" data-b="2085"><span class="d-button deco-button deco-button-borderless d-button_type_borderless d-button_rounded d-button_size_M d-button_w-icon d-button_w-icon-centered d-context-menu__opener d-track__context-menu" type="button"><span class="d-button-inner deco-button-stylable"><span class="d-button__inner"><span class="d-icon deco-icon d-icon_dots-horisontal  "></span></span></span></span></div><div class="d-track__info d-track__nohover"><span class="typo-track deco-typo-secondary">49:02</span></div></div></div></div>

i = 0
while True:
    driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});") 
    i += 1
    sleep(1)
    duration_elements = driver.find_elements_by_xpath('//span[@class="typo-track deco-typo-secondary"]')
    titles.extend([title_elements[j].text for l in range(len(title_elements)) if title_elements[i].text not in titles])

C:\Users\Daria\AppData\Local\Temp/ipykernel_13848/436144416.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  durations = driver.find_elements_by_xpath('//span[@class="typo-track deco-typo-secondary"]')


  0%|          | 0/20 [00:00<?, ?it/s]

49:02
48:44
49:32
49:24
49:33
49:00
48:48
49:10
48:59
48:55
48:38
49:18
48:56
49:03
49:18
49:05
48:56
49:44
49:30
47:27


In [14]:
def get_titles(podcasts_df):
    
    global episodes
    
    for link in podcasts_df['Link']:
        driver.get(link)
        screen_height = driver.execute_script("return window.screen.height;")
        i = 0
        titles = []
        link_index = podcasts_df.index[podcasts_df['Link'] == link][0]
        podcast = podcasts_df['Header'][link_index]
        
        while True:
            driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});") 
            i += 1
            sleep(1)
            title_elements = driver.find_elements_by_xpath('//a[@class="d-track__title deco-link deco-link_stronger line-clamp"]')
            titles.extend([title_elements[i].text for i in range(len(title_elements)) if title_elements[i].text not in titles])
    
            scroll_height = driver.execute_script("return document.body.scrollHeight;")  
            if (screen_height) * i > scroll_height:
                break
        
        for i in range(len(titles)):
            
            ep = {'Title': titles[i], 
                  'Podcast': podcast, 
                  'Duration': None,
                  'Link': None}
                   
            episodes = episodes.append(ep, ignore_index=True)
        
        print(f'{link_index + 1} out of {podcasts_df.shape[0]} done, {len(titles)}/{podcasts_df["Number of Episodes"][link_index]} episodes added')

In [15]:
get_titles(podcasts)

C:\Users\Daria\AppData\Local\Temp/ipykernel_13848/2264719216.py:17: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  title_elements = driver.find_elements_by_xpath('//a[@class="d-track__title deco-link deco-link_stronger line-clamp"]')


1 out of 103 done, 17/17 episodes added


KeyboardInterrupt: 

In [11]:
episodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7511 entries, 0 to 7510
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     7511 non-null   object
 1   Podcast   7511 non-null   object
 2   Duration  0 non-null      object
 3   Link      0 non-null      object
dtypes: object(4)
memory usage: 234.8+ KB


In [26]:
for i in range(len(titles)):
    print(f'{i + 1}. {titles[i]}')

1. Народное инородное: фольклор - раньше и сейчас
2. Геномная селекция: поля прошлого и будущего
3. Японская осень: охота на красные клёны, красивая Луна и хурма
4. Covid-19 и головной мозг: нейроинфекции глазами ученых
5. Что такое популярная "киномакулатура" и как от нее избавиться
6. Азот в атмосфере: стоит ли волноваться, что его так много?
7. Люди пространства и народы времени. Как это происходит?
8. Минимализм: способ спасти планету и себя?
9. Опера как явление и средство изучения жизни
10. Куда дует солнечный ветер?
11. Искусственный интеллект меняет природу человека?
12. Этнографические изыскания. Особенности для собирателей историй
13. Гражданство. Метаморфозы в истории и современности
14. Мифы о животных: где правда, а где вымысел?
15. Историческое языкознание. Как говорили в древности?
16. Вирусы. Темная сторона. Что известно?
17. Почему мы разноцветные? Как ученые объясняют цвет кожи человека.
18. Что мы знаем об эволюции минералов?
19. Парадоксы языка: почему языки такие р

In [11]:
len(set(episodes['Link']))

17

In [12]:
episodes.to_excel('C:/Users/User/Downloads/episodes_v2.xlsx', encoding='utf-8-sig')

In [ ]:
# Drafts

In [60]:
def get_episodes_metadata(podcasts_df):
    
    global episodes
    
    for link in podcasts_df['Link']:

        driver.get(link)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        
        link_index = podcasts_df.index[podcasts_df['Link'] == link][0]
        podcast = podcasts_df['Header'][link_index]
        
        for i in range(podcasts_df['Number of Episodes'][link_index] + 1):
            
            titles = driver.find_elements_by_xpath('//a[@class="d-track__title deco-link deco-link_stronger line-clamp"]')
            durations = driver.find_elements_by_xpath('//span[@class="typo-track deco-typo-secondary"]')
            links = driver.find_elements_by_xpath('//*[contains(@href,"/track/")]')
       
            eps = {'Title': titles[i].text,
                  'Podcast': podcasts,
                  'Duration': durations[i].text, 
                  'Link': links[i].get_attribute('href')}

            episodes = episodes.append(eps, ignore_index=True)
            
        print('1 done')

In [ ]:
def get_episodes_metadata(podcasts_df):
    
    global episodes
    
    for link in podcasts_df['Link']:
        
        link_index = podcasts_df.index[podcasts_df['Link'] == link][0]
        driver.get(link)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        
        for i in range(podcasts_df['Number of Episodes'][link_index]):
            
            podcast = podcasts_df['Header'][link_index]
            titles = driver.find_elements_by_xpath('//div[@class="d-track typo-track d-track_selectable d-track_inline-meta d-track_with-cover d-track_podcast"]/div[@class="d-track__overflowable-column"]/div[@class="d-track__overflowable-wrapper deco-typo-secondary block-layout"]/div[@class="d-track__name"]/a[@class="d-track__title deco-link deco-link_stronger line-clamp"]')
            durations = driver.find_elements_by_xpath('//span[@class="typo-track deco-typo-secondary"]')
       
            ep = {'Title': titles[i].text,
                  'Podcast': podcast,
                  'Duration': durations[i].text}

            episodes = episodes.append(ep, ignore_index=True)
            
        print(f'{podcasts_df.index[podcasts_df["Link"] == link][0] + 1} out of {podcasts_df.shape[0]} done')